In [1]:
import pandas as pd
import tarfile
from tqdm import tqdm
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.utils import to_categorical
from transformers import  BertTokenizer, TFAlbertModel
import implicit
import matplotlib.pyplot as plt
import scipy.sparse as sparse
import tensorflow as tf


In [2]:
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
numpyArray = np.load('../Jiro/features.npy')
np.load = np_load_old
ratings_ = df = pd.DataFrame(numpyArray, columns = ['user_id','business_id','stars','text'])

In [3]:
ratingsnew = ratings_.head(10000)
# ratingsnew["stars"].astype(float).round()

for count in range(len(ratingsnew.text)):
    ratingsnew.at[count, 'text'] = " ".join(ratingsnew.at[count, 'text'])

ratingsnew['text'] = ratingsnew['text'].astype(str)
print(ratings_.head(1000))


                    user_id             business_id stars  \
0    mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   3.0   
1    OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ   5.0   
2    8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A   3.0   
3    _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA   5.0   
4    bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ   4.0   
..                      ...                     ...   ...   
995  syyKcKPFILDysHWmtka-aA  1_hDCN3iioFR3XnUr32ZtA   5.0   
996  _lgLNzpzf3qmbwySBakxEw  5RzJ2bjU8bLSaN5SuiUpYA   4.0   
997  Sh_vUlHHY2Kuj14eF8NYZQ  s1PNBO9o5jIgNd5YWUDLXQ   5.0   
998  YwMD-AVT67fmYRGxnlRSPA  alUk6OwNhofyc90NDMDY-Q   5.0   
999  WKe2b_EeLBnZ3lZV5WKYGQ  -Or44IdY51Ukd618kikmtA   4.0   

                                                  text  
0    [CLS] If you decide to eat here , just be awar...  
1    [CLS] I ' ve taken a lot of spin classes over ...  
2    [CLS] Family diner . Had the b ##uff ##et . E ...  
3    [CLS] Wow ! Yu ##mmy , different ,

C:\Users\apran\AppData\Local\Temp\ipykernel_15040\2454310699.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratingsnew['text'] = ratingsnew['text'].astype(str)


In [4]:
# df_features = df_ratings.pivot_table(
#     index = "user_id",
#     columns = "business_id",
#     values = "labels",
# ).fillna(0)
df_features = ratingsnew
df_features["user_id"]  = df_features['user_id'].astype("category").cat.codes
df_features["business_id"]  = df_features['business_id'].astype("category").cat.codes

C:\Users\apran\AppData\Local\Temp\ipykernel_15040\1422376793.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features["user_id"]  = df_features['user_id'].astype("category").cat.codes
C:\Users\apran\AppData\Local\Temp\ipykernel_15040\1422376793.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features["business_id"]  = df_features['business_id'].astype("category").cat.codes


In [5]:
train_df, test_df = train_test_split(df_features, test_size = 0.2)

y_train = tf.convert_to_tensor(train_df["stars"],dtype = "float")
y_test = tf.convert_to_tensor(test_df["stars"],dtype = "float")

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
albert = TFAlbertModel.from_pretrained('roberta-base')

You are using a model of type roberta to instantiate a model of type albert. This is not supported for all configurations of models and can yield errors.
Some layers from the model checkpoint at roberta-base were not used when initializing TFAlbertModel: ['lm_head', 'roberta']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFAlbertModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['albert']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
x_train = tokenizer(
    text=train_df.text.tolist(),
    add_special_tokens=True,
    max_length=512,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test = tokenizer(
    text=test_df.text.tolist(),
    add_special_tokens=True,
    max_length=512,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [8]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [9]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsolutePercentageError
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [10]:
max_len = 512
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = albert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(1,activation = 'linear')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [11]:
model.layers[2].trainable = False

In [12]:
optimizer = Adam(
    learning_rate=5e-05, 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss = MeanSquaredError()
metric = MeanSquaredError()
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = [MeanSquaredError(),MeanAbsolutePercentageError()])

In [13]:
import datetime
logdir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
  epochs=2,
    batch_size=36
)

Epoch 1/2
223/223 [==============================] - 472s 2s/step - loss: 3.4800 - mean_squared_error: 3.4757 - mean_absolute_percentage_error: 57.5253 - val_loss: 1.8559 - val_mean_squared_error: 1.8546 - val_mean_absolute_percentage_error: 50.4623
Epoch 2/2
223/223 [==============================] - 2524s 11s/step - loss: 1.9180 - mean_squared_error: 1.9176 - mean_absolute_percentage_error: 52.9379 - val_loss: 1.8934 - val_mean_squared_error: 1.8924 - val_mean_absolute_percentage_error: 50.0965


In [14]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]

63/63 [==============================] - 673s 11s/step


array([3.7061822], dtype=float32)

In [ ]:
user_id = test_df['user_id'].tolist()[0]
print(user_id)

2475


In [ ]:
newVector = np.array(predicted_raw[0])
print(newVector)
modelRec.user_factors[user_id]

[3.674292]


NameError: name 'user_id' is not defined

In [ ]:
modelRec.recommend(user_id,data_conf[user_id], N = 20)

(array([2210, 1093,  966,  981, 1754, 1861, 1520, 1628, 2455, 1944, 2094,
        2563, 1154,  343, 2170,  652, 1740,  469, 1185, 1918]),
 array([0.4837332 , 0.43117538, 0.3775813 , 0.36282626, 0.36107212,
        0.3452233 , 0.32818776, 0.3239207 , 0.2970935 , 0.2922436 ,
        0.29059905, 0.28478357, 0.2842365 , 0.28270864, 0.27976733,
        0.278482  , 0.2759957 , 0.27330327, 0.27211142, 0.271969  ],
       dtype=float32))

In [ ]:
modelRec.user_factors[user_id] = newVector
print(modelRec.recommend(user_id,data_conf[user_id], N =20))

(array([2166,  469, 1342,  872, 2613,  252, 2478,  171, 2084, 1810,  666,
        518,  134, 1964,  756,  423,  465,  440,  630,  195, 1827, 2349,
       1185, 2261, 1686,  555, 2491,  107, 1676,   40]), array([0.10468029, 0.09051982, 0.08740012, 0.08347838, 0.08332894,
       0.08098835, 0.08068367, 0.07956488, 0.0774186 , 0.07591844,
       0.07547419, 0.07518977, 0.07424901, 0.0727215 , 0.07023704,
       0.06971195, 0.06905746, 0.06793718, 0.06787977, 0.0671107 ,
       0.06567258, 0.06548437, 0.06544199, 0.06511699, 0.06510273,
       0.06482379, 0.06472807, 0.06373616, 0.0632884 , 0.06257409],
      dtype=float32))


In [17]:
model.save('./albertStar/')

INFO:tensorflow:Assets written to: ./albertStar/assets


INFO:tensorflow:Assets written to: ./albertStar/assets
